# Visualize light curves for all the sources of a region and skymaps of its fields 

In [1]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux

# ------------------  Setup region and plotting paramters  ----------------------

region_name = "TDS" #_ELAISN1
lc_block_srcs = 5 #Setup light curve plotting parameters
show_lc_pars = ["rg_src_id","nr_det","flux","flux_cpval","flux_nxv","flux_rchiq"]
show_map_pars = ["rg_src_id","ra","dec","pos_err","pos_nxv","nr_det","flux","flux_cpval", "nr_fd_srcs"]
sort_par = "rg_src_id" # "flux" #"rg_src_id" #
only_sel_srcs = True

# -------------------------------------------------------------------------------

#Load region
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
rg = Region()
rg.load_from_fits(region_fname)

tt_srcs = rg.tt_sources
if only_sel_srcs:
    tt_srcs = rg.tt_sources[rg.tt_sources["sel"]]
nr_srcs = len(tt_srcs)
tt_srcs.sort(sort_par)
tt_srcs.add_index("rg_src_id")

### SHow all info for one source

In [2]:
def plot_sky_src(rg_src_id=0):
    if rg_src_id not in tt_srcs["rg_src_id"]:
        print("source ID does not exist") 
        return
    
    fig_src = plt.figure("Sky map source",figsize=(8, 7))
    plt.cla()

    tc_src = rg.get_src_from_id(rg_src_id)
    
    #Get sky region of source
    sky_center = SkyCoord(tc_src.tt_sources[0]["ra"]*uu.deg, tc_src.tt_sources[0]["dec"]*uu.deg, frame="icrs")
    sky_region_src = CircleSkyRegion(center=sky_center, radius=0.5*uu.arcmin)
    
    #Get field to plot
    fd_src = rg.get_field(rg_fd_id=tc_src.tt_fields[0]["rg_fd_id"], load_method="FITS", add_field=True)
    
    #Plot
    #plt.cla()
    ax_src, wcs_src = vvis.plot_field_sky_map(fd_src, img_idx=-1, sky_region = sky_region_src,fig=fig_src) # 
    _ = vvis.plot_sky_sources(tc_src.tt_sources,tt_det=tc_src.tt_detections, sky_region_wcs = (sky_region_src,wcs_src),only_selected=False) #
    plt.show()
    
    fig_lc_src = plt.figure("Light curve",figsize=(8, 6))
    plt.cla()
    vvis.plot_light_curve(rg,fig=fig_lc_src, rg_src_ids = rg_src_id)
    plt.show()
    
    
    display(tc_src.tt_sources[show_map_pars])
    display(tc_src.tt_detections)
    display(tc_src.tt_fields)
widgets.interactive( plot_sky_src, rg_src_id=tt_srcs["rg_src_id"])

interactive(children=(Dropdown(description='rg_src_id', options=(2, 6, 145, 148, 154, 222, 271, 498, 555, 564,…

### Draw sky maps of fields

In [3]:
def plot_sky_field(rg_fd_id):
    if rg_fd_id not in rg.tt_fields["rg_fd_id"]:
        print("source ID does not exist") 
        return
    fig_skymap = plt.figure("Sky map",figsize=(9, 8)) #, constrained_layout=True
    plt.cla()
    fd = rg.get_field(rg_fd_id=rg_fd_id, load_method="FITS", add_field=True)
    ax, wcs = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = fd.get_sky_region(),fig=fig_skymap) # 
    ax, rg_src_ids = vvis.plot_sky_sources(tt_srcs,tt_det=rg.tt_detections, sky_region_wcs = (fd.get_sky_region(),wcs),only_selected=False) # 
    src_idx = tt_srcs.loc_indices["rg_src_id", rg_src_ids]
    plt.show()
    display(tt_srcs[src_idx][show_map_pars])
widgets.interactive( plot_sky_field, rg_fd_id=(rg.tt_fields["rg_fd_id"].min(),rg.tt_fields["rg_fd_id"].max()))

interactive(children=(IntSlider(value=20, description='rg_fd_id', max=41), Output()), _dom_classes=('widget-in…

### Draw light curves

In [4]:
nr_blocks = int(nr_srcs/lc_block_srcs)+1 
flux_ylim = mag2flux([24.5,14.5])/1e-6
def plot_lc_range(block_nr):
    fig_lc = plt.figure("Light curves",figsize=(10, 9))
    plt.cla()
    lc_min = lc_block_srcs * block_nr
    lc_max = lc_min + lc_block_srcs if lc_min + lc_block_srcs<=nr_srcs-1 else nr_srcs-1
    vvis.plot_light_curve(rg,fig=fig_lc, rg_src_ids = tt_srcs["rg_src_id"][lc_min:lc_max],ylim = flux_ylim.value ) #, 
    plt.show()
    display(tt_srcs[lc_min:lc_max][show_lc_pars])
widgets.interactive( plot_lc_range, block_nr=(0,nr_blocks-1))

interactive(children=(IntSlider(value=186, description='block_nr', max=373), Output()), _dom_classes=('widget-…